In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle


In [53]:
# Load the model , pickle files

model=load_model('model.h5')

with open ('gender_value.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('geo_value.pkl','rb') as file:
    one_hot_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [99]:
label_encoder_gender.classes_
one_hot_geo.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [66]:
input_data= {    
    'CreditScore' : 600,
    'Gender': 'Male',
    'Age':40,
    'Tenure':3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Geography': 'France'

}

In [28]:
one_hot_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [100]:
geo_encoded=one_hot_geo.transform([[input_data['Geography']]]).toarray()

geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_geo.get_feature_names_out())

geo_encoded_df

ValueError: Found array with dim 3. None expected <= 2.

In [67]:
input_data

{'CreditScore': 600,
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000,
 'Geography': 'France'}

In [68]:
input_data=pd.DataFrame([input_data])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,Male,40,3,60000,2,1,1,50000,France


In [45]:
pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,France,1.0,0.0,0.0


In [43]:
input_data.reset_index(drop=True)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,Male,40,3,60000,2,1,1,50000,France


In [69]:
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])


In [70]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,600,1,40,3,60000,2,1,1,50000,France


In [80]:
input_df=pd.concat([input_data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [81]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
input_scaled=scaler.transform(input_df)


In [87]:
prediction=model.predict(
    input_scaled
)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[0.09017045]], dtype=float32)

In [92]:
prediction_proba=prediction[0][0]
prediction_proba

0.09017045

In [93]:
if prediction_proba > 0.5:
    print("Yes customer gonna churn")
else:
    print("Customer is NOT likely to churn")

Customer is NOT likely to churn
